# Homework 1

#### Nawat Ngerncham

In [ ]:
import matplotlib.pylab as plt
import seaborn as sns

from experiment import run_experiment

## GR17

In [ ]:
%%time
gr17_result = run_experiment("data/gr17_d.txt", [], 1500)

In [ ]:
gr17_result.to_csv("results/gr17.txt")
gr17_result

In [ ]:
%%time
sns.lineplot(gr17_result, x="Iterations", y="Distance", hue="Type")
plt.axhline(2085, color="black")
plt.savefig("report/images/gr17_plot.png", dpi=1200, bbox_inches="tight")
plt.show()

# FRI26


In [ ]:
%%time
fri26_result = run_experiment("data/fri26_d.txt", [], 2500)

In [ ]:
fri26_result.to_csv("results/fri26.txt")
# fri26_result

In [ ]:
%%time
sns.lineplot(fri26_result, x="Iterations", y="Distance", hue="Type")
plt.axhline(937, color="black")
plt.savefig("report/images/fri26_plot.png", dpi=1200, bbox_inches="tight")
plt.show()

# ATT48

In [ ]:
%%time
att48_result = run_experiment("data/att48_d.txt", [], 10_000)

In [ ]:
att48_result.to_csv("results/att48.txt")
# att48_result

In [ ]:
%%time
sns.lineplot(att48_result, x="Iterations", y="Distance", hue="Type")
plt.axhline(33523, color="black")
plt.savefig("report/images/att48_plot.png", dpi=1200, bbox_inches="tight")
plt.show()

# Running Time

In [ ]:
from experiment import run_timer

run_timer("data/gr17_d.txt", [], 2500)

In [ ]:
run_timer("data/fri26_d.txt", [], 2500)

In [ ]:
run_timer("data/att48_d.txt", [], 2500)